## Imports and Data Cleaning

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import torch 
from torch import nn
from torch import optim 

pd.options.mode.chained_assignment = None

/Users/nickmelamed/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("./data/bank-additional-full.csv", delimiter = ";")
df.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


The description of the columns can be found at the UCI Machine Learning Repository, linked [here](https://archive.ics.uci.edu/dataset/222/bank+marketing)

In [3]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

For this particular analysis, I am going to ignore economic factors, as I will take the simplifying assumption that the majority of people who are in this campaign are not making decisions based on economic factors like the consumer price index.

In [4]:
df = df[df.columns[~df.columns.isin(['emp.var.rate', 'cons.price.idx','cons.conf.idx', 'euribor3m', 'nr.employed'])]]
df.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


In [5]:
df['default'].value_counts()

no         32588
unknown     8597
yes            3
Name: default, dtype: int64

In [6]:
df['housing'].value_counts()

yes        21576
no         18622
unknown      990
Name: housing, dtype: int64

In [7]:
df['loan'].value_counts()

no         33950
yes         6248
unknown      990
Name: loan, dtype: int64

As we can see from above, the number of customers who have defaulted on previous loans is relatively low. So, to avoid dealing with the unknown values, we will make the assumption that the vasy majority of people have not defaulted and therefore this column does not provide us any predictive information. 

We will also drop the unknowns from people who have taken housing or personal loans, as we have a sufficient sample size without the unknowns.

In [8]:
df = df.drop(columns = ['default'])

df = df[(df['housing'] != 'unknown') & (df['loan'] != 'unknown')]

In order to run our neural network, we have to make all of our variables numeric. We do so by turning any categorical variables into dummy variables (1 = variable is true, 0 = false), and, since the binary variables are stored as yes/no variables, we will convert those into 1/0 values as well. 

In [9]:
# converting categorical variables into dummies

df_dummy = pd.get_dummies(df, columns = ['job', 'marital', 'education', 'contact', 'month', 'day_of_week', 'poutcome'], prefix_sep = ': ')

# turning y/n variables into 1/0

df_dummy['housing'] = np.where(df['housing'].values == 'yes', 1, 0)
df_dummy['loan'] = np.where(df['loan'].values == 'yes', 1, 0)
df_dummy['y'] = np.where(df['y'].values == 'yes', 1, 0)

df_dummy.head(5)

,age,housing,loan,duration,campaign,pdays,previous,y,job: admin.,job: blue-collar,...,month: oct,month: sep,day_of_week: fri,day_of_week: mon,day_of_week: thu,day_of_week: tue,day_of_week: wed,poutcome: failure,poutcome: nonexistent,poutcome: success
0,56,0,0,261,1,999,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,57,0,0,149,1,999,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,37,1,0,226,1,999,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,40,0,0,151,1,999,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
4,56,0,1,307,1,999,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


## Building the Neural Network

For any model building, we want to ensure our model is not trained on the entirety of our dataset. Otherwise, we would have no data to test the model on, and run the risk of overtraining. I have decided to use Scikit-Learn to split the dataset into 70% training data, 30% testing data. An 80-20 split is another popular option but I prefer to have more testing data to ensure the model is even less susceptible to the overtraining problem.

We make sure our outcome variable, y (whether or not the client subscribed to the term deposit, the goal of the campaign), is separated from our other predictor variables.  

In [10]:
X = df_dummy[df_dummy.columns[~df_dummy.columns.isin(['y'])]]
y = df_dummy['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .70)

Now that we have the data split up, we want to build our model and train it on the training data. We use PyTorch because of the ease of converting data into tensors, and using tensors as inputs to models. 

In [11]:
# converting DF into Tensor

X_train_np = X_train.to_numpy() 
y_train_np = y_train.to_numpy()

X_train_tensor = torch.from_numpy(X_train_np).to(torch.float32) # convert to float32 b/c nn.Sequential uses this dtype
y_train_tensor = torch.from_numpy(y_train_np).reshape(-1, 1).to(torch.float32)

In [12]:
# Building the model using PyTorch

num_inputs = X_train_tensor.shape[1]

model = nn.Sequential(
    nn.Linear(num_inputs, 60),
    nn.ReLU(),
    nn.Linear(60, num_inputs),
    nn.ReLU(),
    nn.Linear(num_inputs, 1),
    nn.Sigmoid())

Since we are using a binary classifier (either the campaign is successful or it isn't), we use Binary Cross Entropy loss as it measures the error in mislabeled outcomes. We use the Adam optimizer due to its ability to have quick convergence and deal with sparse gradients, which we may deal with as a result of having a lot of dummy variables. 

In [13]:
loss_fn = nn.BCELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:
# training the model using 100 epochs, each with batch size of 10 

n_epochs = 100
batch_size = 10
 
for epoch in range(n_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        Xbatch = X_train_tensor[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y_train_tensor[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 0.13836824893951416
Finished epoch 1, latest loss 0.10167008638381958
Finished epoch 2, latest loss 0.11523856222629547
Finished epoch 3, latest loss 0.12235945463180542
Finished epoch 4, latest loss 0.09294992685317993
Finished epoch 5, latest loss 0.09439419209957123
Finished epoch 6, latest loss 0.09099025279283524
Finished epoch 7, latest loss 0.07929471135139465
Finished epoch 8, latest loss 0.07711247354745865
Finished epoch 9, latest loss 0.08769702166318893
Finished epoch 10, latest loss 0.09161672741174698
Finished epoch 11, latest loss 0.0711621567606926
Finished epoch 12, latest loss 0.07862813770771027
Finished epoch 13, latest loss 0.07979511469602585
Finished epoch 14, latest loss 0.06433399021625519
Finished epoch 15, latest loss 0.07841551303863525
Finished epoch 16, latest loss 0.09051502496004105
Finished epoch 17, latest loss 0.07747684419155121
Finished epoch 18, latest loss 0.07898355275392532
Finished epoch 19, latest loss 0.063042782

We compute the training accuracy of our model to determine how well it classifies our desired outcome.

In [21]:
with torch.no_grad():
    y_pred_train = model(X_train_tensor)

train_accuracy = (y_pred_train.round() == y_train_tensor).float().mean()
print(f"Training Accuracy: {round(float(train_accuracy)*100, 2)}%") # have to convert accuracy from Tensor to float

Training Accuracy: 90.68%


This is a very encouraging training accuracy! However, it is quite high and could be a result of overtraining our data. 

To ensure this isn't the case, we will use our test data and see how the model performs on this set of data: 

In [20]:
# Converting test data to Tensors

X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()

X_test_tensor = torch.from_numpy(X_test_np).to(torch.float32)
y_test_tensor = torch.from_numpy(y_test_np).reshape(-1, 1).to(torch.float32)

In [23]:
# computing test accuracy 

with torch.no_grad():
    y_pred_test = model(X_test_tensor)

test_accuracy = (y_pred_test.round() == y_test_tensor).float().mean()
print(f"Test Accuracy: {round(float(test_accuracy)*100, 2)}%")

Test Accuracy: 90.43%


We have a similarly high test data accuracy, suggesting we are not dealing with an overtraining problem. 